In [ ]:
# Cell 1: imports & config
import os
from pathlib import Path
import time
import threading
from dotenv import load_dotenv

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam

import cv2
import schedule
import smtplib
from email.message import EmailMessage

# load environment variables
load_dotenv()
EMAIL_USER = os.getenv("EMAIL_USER")
EMAIL_PASS = os.getenv("EMAIL_PASS")
CAREGIVER_EMAIL = os.getenv("CAREGIVER_EMAIL")


In [ ]:
# Cell 3: data generators
DATA_DIR = Path("data")  # expect data/train and data/val inside
IMG_SIZE = (224,224)
BATCH = 16

train_dir = str(DATA_DIR / "train")
val_dir = str(DATA_DIR / "val")

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(train_dir,
                                              target_size=IMG_SIZE,
                                              batch_size=BATCH,
                                              class_mode='categorical')

val_gen = val_datagen.flow_from_directory(val_dir,
                                          target_size=IMG_SIZE,
                                          batch_size=BATCH,
                                          class_mode='categorical')
class_indices = train_gen.class_indices
print("Classes:", class_indices)
